# Deep dive into Pandas DataFrames 

**Read the official documentation on pandas DataFrames @ https://pandas.pydata.org/pandas-docs/stable/reference/frame.html**

**`Note:`** The notion of **chaining functions/methods** in pandas is similar to python.

DataFrames are **column oriented** unlike most common databases. And, **each column** in the dataframe is a **pandas series object**. So, any operation that can be performed on a pandas series object it can be applied to a column too.

There are **two axes** for a dataframe commonly referred to as axis 0 and 1, or the **"index"** (or 'rows') axis and the **"columns"** axis respectively. Note that, when an **operation** is applied **along axis 0**, it is applied **down the column**. Likewise, operations **along axis 1** operate **across the values in the row**.

--------------------
### Import Statements
--------------------------

In [1]:
# import statements
import numpy as np
import pandas as pd

In [2]:
# view options
pd.set_option("display.max_columns", 40)
pd.set_option("display.max_rows", 10)

---------------------------
### Importing the data
------------------------

We will be exploring a dataset from a Siena College Poll in 2018. This data has rankings of United States Presidents in various attributes. These attributes are:

In [3]:
# reading from github url

# it is a good practice to define your index column when reading the data file.
# it is generally frowned upon if you don't have an index column

url = "https://github.com/mattharrison/datasets/raw/master/data/siena2018-pres.csv"
siena_2018 = pd.read_csv(url, index_col=0)

In [4]:
siena_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 1 to 44
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Seq.       44 non-null     object
 1   President  44 non-null     object
 2   Party      44 non-null     object
 3   Bg         44 non-null     int64 
 4   Im         44 non-null     int64 
 5   Int        44 non-null     int64 
 6   IQ         44 non-null     int64 
 7   L          44 non-null     int64 
 8   WR         44 non-null     int64 
 9   AC         44 non-null     int64 
 10  EAb        44 non-null     int64 
 11  LA         44 non-null     int64 
 12  CAb        44 non-null     int64 
 13  OA         44 non-null     int64 
 14  PL         44 non-null     int64 
 15  RC         44 non-null     int64 
 16  CAp        44 non-null     int64 
 17  HE         44 non-null     int64 
 18  EAp        44 non-null     int64 
 19  DA         44 non-null     int64 
 20  FPA        44 non-null     int64 


In [5]:
siena_2018_cols = """
• Bg = Background
• Im = Imagination
• Int = Integrity
• IQ = Intelligence
• L = Luck
• WR = Willing to take risks
• AC = Ability to compromise
• EAb = Executive ability
• LA = Leadership ability
• CAb = Communication ability
• OA = Overall ability
• PL = Party leadership
• RC = Relations with Congress
• CAp = Court appointments
• HE = Handling of economy
• EAp = Executive appointments
• DA = Domestic accomplishments
• FPA = Foreign policy accomplishments
• AM = Avoid crucial mistakes
• EV = Experts’ view
• O = Overall
"""

---------------
## Mathematical operations on DataFrames
----------------

**Similar to series objects, Math operations for DataFrames are Index Aligned.**

Aligning will take each index entry from a particular column in the left df and match it up with every entry with the same index of the same column in the right df. This is repeated for all the overlapping columns. If any of the df has duplicate index this will cause the addition operation to behave unexpectedly i.e, it will work by process of permutating the matching indexex.

In [6]:
# s1: 3 rows and 4 columns
# s2: 2 rows and 5 columns
s1 = pd.DataFrame(
    np.linspace(2, 13, 12).reshape(3, 4),
    columns=["a1", "b1", "c1", "d1"],
    index=[1, 2, 3],
)
s2 = pd.DataFrame(
    np.linspace(2, 11, 10).reshape(2, 5),
    columns=["a1", "b1", "c1", "d1", "e1"],
    index=[2, 2],
)

In [7]:
s1 + s2

,a1,b1,c1,d1,e1
1,NaN,NaN,NaN,NaN,NaN
2,8.0,10.0,12.0,14.0,NaN
2,13.0,15.0,17.0,19.0,NaN
3,NaN,NaN,NaN,NaN,NaN


As we can see, only the **overlapping rows** (2nd row) **and columns** (a1 through d1) get added together. The other values are missing. We can use the **.add method instead of "+" and define a fill value** if we wanted, similar to what we've done in case of series objects.

In [8]:
s1.add(s2, fill_value=0)

,a1,b1,c1,d1,e1
1,2.0,3.0,4.0,5.0,NaN
2,8.0,10.0,12.0,14.0,6.0
2,13.0,15.0,17.0,19.0,11.0
3,10.0,11.0,12.0,13.0,NaN


---------------------------------------------------
## Looping over a DataFrame (using the `for` loop)
-----------------------------------------------------

It is generally not a good practice and is usually frowned upon if you use for loop with your pandas dataframe. This is because, pandas built-in methods are much faster than for loop due to vectorization and you are not taking advantage of it. However some times it might be useful to use for loop in datafrmes such as, when plotting visuals.

Some iterator methods that are useful while looping over a dataframe are, **.items(), .iterrows(), .itertuples()**.

- The `.items()` method: returns a tuple of **(column name, column content as Series)**

The indexes of the returned series will be the indexes of the dataframe.

In [9]:
for col_label, col_content in siena_2018.items():
    print(f"Column Name: {col_label}")
    print(f"Data contents of the column: \n{col_content}")
    break

Column Name: Seq.
Data contents of the column: 
1      1
2      2
3      3
4      4
5      5
      ..
40    41
41    42
42    43
43    44
44    45
Name: Seq., Length: 44, dtype: object


- The `.iterrows()` method: returns a tuple of **(index, row content as Series)**

The indexes of the returned series will be the associated column names.

In [10]:
for idx, row_content in siena_2018.iterrows():
    print(f"Row index: {idx}\n")
    print(f"Data contents of the row: \n\n{row_content}")
    break

Row index: 1

Data contents of the row: 

Seq.                         1
President    George Washington
Party              Independent
Bg                           7
Im                           7
                   ...        
DA                           2
FPA                          2
AM                           1
EV                           2
O                            1
Name: 1, Length: 24, dtype: object


- The `.itertuples()` method: returns the **rows as namedtuples**

In [11]:
for row in siena_2018.itertuples():
    print(row)
    break

Pandas(Index=1, _1='1', President='George Washington', Party='Independent', Bg=7, Im=7, Int=1, IQ=10, L=1, WR=6, AC=2, EAb=2, LA=1, CAb=11, OA=2, PL=18, RC=1, CAp=1, HE=1, EAp=1, DA=2, FPA=2, AM=1, EV=2, O=1)


--------------------------------
## Aggregations
--------------------------

Aggregations that are applicable to a Series object are also applicable to a DataFrame. The only difference is that the aggregations can be applied across 2 axis (i.e, index and columns).

In [12]:
# let's slice out a portion from the siena_2018 df that has numerical values it's good practice to use .copy()
# while slicing a df so that operations applied on the sliced df doesn't affect the original df
scores = siena_2018.loc[:, "Bg":"O"].copy()

#### Multiple aggregations on a dataframe using the `.agg` method

- **Aggregate over axis 1 (apply function to each row i.e, aggregate across the columns)**

In [13]:
scores.agg(["sum", "mean"], axis=1).tail(3)

,sum,mean
42,635.0,30.238095
43,331.0,15.761905
44,833.0,39.666667


- **Aggregate over axis 0 (apply function to each column i.e, aggregate across the rows)**

In [14]:
scores.agg(["sum", "mean"], axis=0).head(3)

,Bg,Im,Int,IQ,L,WR,AC,EAb,LA,CAb,OA,PL,RC,CAp,HE,EAp,DA,FPA,AM,EV,O
sum,968.0,957.00,990.0,990.0,990.0,953.000000,968.0,978.000000,990.0,990.0,990.0,990.0,979.00,990.0,990.0,990.0,990.0,990.0,990.0,990.0,990.0
mean,22.0,21.75,22.5,22.5,22.5,21.659091,22.0,22.227273,22.5,22.5,22.5,22.5,22.25,22.5,22.5,22.5,22.5,22.5,22.5,22.5,22.5


- Different aggregations per column

In [15]:
scores.agg({"Int": ["max", "mean"], "IQ": ["min", "mean"]})

,Int,IQ
max,44.0,NaN
mean,22.5,22.5
min,NaN,1.0


#### The `.describe` returns a dataframe with summary statistics for each numeric columns

In [16]:
scores.describe()

,Bg,Im,Int,IQ,L,WR,AC,EAb,LA,CAb,OA,PL,RC,CAp,HE,EAp,DA,FPA,AM,EV,O
count,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
mean,22.000000,21.750000,22.500000,22.500000,22.500000,21.659091,22.000000,22.227273,22.500000,22.500000,22.500000,22.500000,22.250000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000
std,12.409674,12.519984,12.845233,12.845233,12.845233,11.892822,12.409674,12.500909,12.845233,12.845233,12.845233,12.845233,12.519984,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,11.750000,11.000000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000
50%,22.000000,21.500000,22.500000,22.500000,22.500000,22.500000,22.000000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000
75%,32.250000,32.250000,33.250000,33.250000,33.250000,31.250000,32.250000,32.250000,33.250000,33.250000,33.250000,33.250000,33.000000,33.250000,33.250000,33.250000,33.250000,33.250000,33.250000,33.250000,33.250000
max,43.000000,43.000000,44.000000,44.000000,44.000000,41.000000,43.000000,43.000000,44.000000,44.000000,44.000000,44.000000,43.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000


**Note:** The count row in the summary statistics has a particular meaning in pandas. It is not the count of the rows, rather it is the count of the non-missing (not na) rows.

---------------------
## Casting Datatypes and Renaming the columns
-----------------------

**Note: This (casting datatypes and renaming columns) should be the first step whenever we load in a dataset. Also, we should write these commands as functions, allowing us to reuse the code in other notebooks if necessary.**

### Renaming the columns with proper full form

- Getting the full form of each column from the "siena_2018_cols" string

In [17]:
# we want to write a code to generate a python dictionary from the above multiline string named "siena_2018_cols", which is
# formatted as short form = long form. This dictionary will be used to rename the columns of the dataframe "siena_2018"

# first we create a list of the form, [[short, full], .....]
cols_list = [
    col.strip().split("=") for col in siena_2018_cols.strip().split(sep="•")[1:]
]

# we will replace the spaces in the full form with underscores (_)
siena_2018_cols_dict = {
    col_prev.strip(): col_full.strip().replace(" ", "_")
    for col_prev, col_full in cols_list
}

**Note:** When such unpacking pattern is used with the for loop in a nested list, it will start to unpack from the most inner layer and not the outer one.

#### The `.rename()` method

In [18]:
# inplace = True is frowned upon
siena_2018 = siena_2018.rename(columns={"Seq.": "Seq"}).rename(
    columns=siena_2018_cols_dict
)

In [19]:
siena_2018.head(3)

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall
1,1,George Washington,Independent,7,7,1,10,1,6,2,2,1,11,2,18,1,1,1,1,2,2,1,2,1
2,2,John Adams,Federalist,3,13,4,4,24,14,31,21,21,13,8,28,17,4,13,15,19,13,16,10,14
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,6,6,4,4,5,5,7,20,4,6,9,7,5,5


### Casting DataTypes

The first thing we should do when we load in a dataset is checking the datatypes of each column and converting each of them to datatypes that is more suitable for them. This will save space and will increase the overall speed of all the operations.

In [20]:
siena_2018.dtypes.to_dict()  # we could've also used the .info() method

{'Seq': dtype('O'),
 'President': dtype('O'),
 'Party': dtype('O'),
 'Background': dtype('int64'),
 'Imagination': dtype('int64'),
 'Integrity': dtype('int64'),
 'Intelligence': dtype('int64'),
 'Luck': dtype('int64'),
 'Willing_to_take_risks': dtype('int64'),
 'Ability_to_compromise': dtype('int64'),
 'Executive_ability': dtype('int64'),
 'Leadership_ability': dtype('int64'),
 'Communication_ability': dtype('int64'),
 'Overall_ability': dtype('int64'),
 'Party_leadership': dtype('int64'),
 'Relations_with_Congress': dtype('int64'),
 'Court_appointments': dtype('int64'),
 'Handling_of_economy': dtype('int64'),
 'Executive_appointments': dtype('int64'),
 'Domestic_accomplishments': dtype('int64'),
 'Foreign_policy_accomplishments': dtype('int64'),
 'Avoid_crucial_mistakes': dtype('int64'),
 'Experts’_view': dtype('int64'),
 'Overall': dtype('int64')}

> **First, let's explore the columns with "Object" datatype**

- The "Seq" column (Sequences of the presidency)

In [21]:
siena_2018.Seq.values

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22/24',
       '23', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45'],
      dtype=object)

Upon inspection we can see that, there's a value of '22/24'. So this column has to remain as "Object" type (or, can be converted to "string" type). 

- The "President" column lists the name of the president. So, this can either be converted to "string" type or can remain as is. 

- The "Party" column provides the name of the party, the president was elected with

In [22]:
siena_2018.Party.value_counts()

Republican               19
Democratic               15
Democratic-Republican     4
Whig                      3
Independent               2
Federalist                1
Name: Party, dtype: int64

This column has only 6 unique values. So, this can be converted to "categorical" type.

In [23]:
siena_2018 = siena_2018.astype({"Party": "category"})

> **Now, let's explore the columns with "int64" as datatype**

**Note:** One of the interesting and important pandas methods is the `.select_dtypes()` method. This will select all the columns with the specified datatype and return those columns as a new DataFrame.

In [24]:
siena_2018.select_dtypes("int64").head(3)

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall
1,7,7,1,10,1,6,2,2,1,11,2,18,1,1,1,1,2,2,1,2,1
2,3,13,4,4,24,14,31,21,21,13,8,28,17,4,13,15,19,13,16,10,14
3,2,2,14,1,8,5,14,6,6,4,4,5,5,7,20,4,6,9,7,5,5


- Let's see the max and min values of the number type columns

In [25]:
siena_2018.select_dtypes("int64").agg(["max", "min"])

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall
max,43,43,44,44,44,41,43,43,44,44,44,44,43,44,44,44,44,44,44,44,44
min,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


As we can see, none of the columns has values greater than 44 and lesser than 1. So, these columns can easily be converted to "uint8" type and still accomodate the values as is.

In [26]:
siena_2018 = siena_2018.astype(
    {col_name: "uint8" for col_name in siena_2018.select_dtypes("int64").columns}
)

After casting datatypes to more appropriate types, the memory footprint of the dataframe reduces drastically.

In [27]:
siena_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 1 to 44
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Seq                             44 non-null     object  
 1   President                       44 non-null     object  
 2   Party                           44 non-null     category
 3   Background                      44 non-null     uint8   
 4   Imagination                     44 non-null     uint8   
 5   Integrity                       44 non-null     uint8   
 6   Intelligence                    44 non-null     uint8   
 7   Luck                            44 non-null     uint8   
 8   Willing_to_take_risks           44 non-null     uint8   
 9   Ability_to_compromise           44 non-null     uint8   
 10  Executive_ability               44 non-null     uint8   
 11  Leadership_ability              44 non-null     uint8   
 12  Communication_ability   

-------------------------------------------------
## Creating and Updating columns: The `.assign(**kwargs)` method
---------------------------------------------------

**Why use .assign ?** This method returns a dataframe and doesn't mutate the existing dataframe. This is very useful for chaining operations as the dataframe gets continuously updated and the subsequent methods operates on the updated dataframe.

<u>**\*\*kwargs:** argument (column name) = argument value (callable or Series}, ...... </u>
- if the column already exists it will modify the values of the column
- if the column doesn't exist then it will create a new column
- if the argumnent value is a series or a scalar, it will simply assign those values to the column
- the callable (a function or *lambda*) must return a scalar or series. Using a function (it can be a normal function, but often we use a lambda to have the logic inline) has an unseen benefit. If any manipulation of filtering was done on the dataframe before using the `.assign()`, those changes will be represented on the dataframe and *the function will accept the current state of the dataframe.*

**`lambda` function Refresher:** A lambda function can take any number of arguments, but can only have one expression. *Syntax:* `lambda arguments : expression`. The expression is executed and the result is returned.

In [28]:
# First, we will add a column named Average_rank that ranks the presidents based on their toatal score (summing the numeric values across the columns)
# using dense method (lowest rank in the group but rank always increases by 1 between groups)
# this is essentially the "Overall" column but using a different ranking method

# Next, we will add another column named, "Quartile_rank" that will have 4 bins (1st, 2nd, 3rd, 4th)
# this is when we will see the power of using a function 
# the lambda function will take the current state of the dataframe when the Average_rank column exists

siena_2018 = siena_2018.assign(Average_rank=siena_2018.loc[:, "Background":"Experts’_view"].sum(axis=1).rank(method="dense").astype("uint8"), 
Quartile_rank=lambda df_: pd.qcut(df_.Average_rank, 4, labels=["1st", "2nd", "3rd", "4th"]))

In [29]:
siena_2018.head(3)

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall,Average_rank,Quartile_rank
1,1,George Washington,Independent,7,7,1,10,1,6,2,2,1,11,2,18,1,1,1,1,2,2,1,2,1,1,1st
2,2,John Adams,Federalist,3,13,4,4,24,14,31,21,21,13,8,28,17,4,13,15,19,13,16,10,14,13,2nd
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,6,6,4,4,5,5,7,20,4,6,9,7,5,5,5,1st


--------------------------------------------
## Dealing with Missing and Duplicated Data
----------------------------------------------

### Locating missing data

- The `.isna()` method

Works similarly to series.isna() method. Returns a Boolean dataframe when used with dataframes.  

In [30]:
siena_2018.isna().head(3)

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall,Average_rank,Quartile_rank
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


We can use other methods such as **.any(), .all() etc.** in combination with the .isna() method to see whether there are any data at all missing from a column or whether all data in a column is Nan type etc.

In [31]:
siena_2018.isna().any().head(3)

Seq          False
President    False
Party        False
dtype: bool

To `count` how many data rows are `missing` from a particular column we can use the **df.isna().sum()**.

In [32]:
siena_2018.isna().sum().head(3)

Seq          0
President    0
Party        0
dtype: int64

To see what `percentage` of the data in a column is missing we can use something like, **df.isna().mean().mul(100)**.

In [33]:
siena_2018.isna().mean().mul(100).sample(3)

Intelligence             0.0
Party_leadership         0.0
Willing_to_take_risks    0.0
dtype: float64

### Handling missing values

- The `.dropna(subset)` method 

We can use the good old .dropna() method to drop the rows with missing values. But note that, when using with dataframe, .dropna() will only drop the rows if it has Nan values in all the columns. To specify otherwise i.e, what columns to look at when dropping rows, we can feed the subset parameter a list of column names (that we want it to look at for dropping Nan values).

- The `.fillna()` method

We can also use the .fillna() method to fill in the missing value. We can also define the filling method e.g, `ffill`, `bfill` etc. This method also takes a `value parameter` (value: scalar, dict, Series, or DataFrame) which will be used to fill the Nan values if specified. The **.mean(), .median(), .mode()** etc methods may come in handy when defining the value paramether.

- The `.interpolate()` method

This will replace Nan values with interpolation of the values around the missing value. This method comes in very handy when dealing with ordered data such as time series data.

- The `.where(cond, other)` method

Although not specific for handling missing values but this method is a powerful one for doing just that. This method **replaces values where the condition is False with corresponding value from 'other'**.

- The `.mask(cond, other)` method

Opposite of the .where() method in the sense that, this method will replace values **where the condition is True** with the corresponding value from 'other'. Equivalent to, **.where(~cond, other)**.

**`CAUTION:`** The data in each column of a dataframe usually represents different things. Thus applying methods such as .dropna(), .fillna(), .interpolate() is not logical and will bring no good (this is like, using a spoon for woodworking).

**So, the best approach is to treat each column differently as a separate series object, clean them, modify them and then adding/replacing them in a datafrmae using the .assign() method.**

### Handling duplicate data

- The `.drop_duplicates(subset=None,keep='first', ignore_index=False)` method

If called without any parameters, it will drop only the rows that are complete copy of each other. The subset parameter lets us specify which columns to check when checking for duplicates.

--------------------------------------------
## Sorting Columns and Indexes 
---------------------------------------------

#### Setting indexes: The `.set_index()` method

Return dataframe with the new index.

<u> Parameters -- </u>

- **keys**: column(s) to be set as index.
- **drop = True** : default True. Indicates whether to remove columns used for the index.
- **verify_integrity = False** : check for duplicate index values by setting verify_integrity=True.

#### Sorting indexes: The `.sort_index()` method

<u> Parameters -- </u>

- **axis = 0**: This method will return dataframe with index (axis=0) or columns (axis=1) sorted.  
- **ascending = True**: default True.
- **key = None**:  A key function accepts an index and should return an index. For multi-level indexes, each index is passed in independently to the function.

This operation is usually done after setting a new index. If the new index is of **string type** then **sorting it will allow us to use slicing** operation on the index column. Othrwise it will throw a KeyError.

#### Sorting values: The `.sort_values()` method

<u> Parameters -- </u>

- **by**: column name or a list of names to sort by.
- **ascending = True**: bool or list of bool, default True.
- **key = None**: Apply the key function to the values before sorting. This is similar to the `key` argument in the builtin **sorted** function. A key function accepts a series and should return a series with the same index.


In [49]:
siena_2018.sort_values(by=["Quartile_rank", "Intelligence"], ascending=[True, False]).sample(3)

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall,Average_rank,Quartile_rank
31,32,Franklin D. Roosevelt,Democratic,6,3,16,12,5,3,4,3,3,2,3,1,3,2,2,3,3,1,4,3,2,2,1st
29,30,Calvin Coolidge,Republican,32,36,17,33,13,39,27,32,38,37,33,26,24,31,24,32,33,35,22,32,31,31,3rd
39,40,Ronald Reagan,Republican,27,17,24,31,3,13,10,15,7,6,18,4,6,18,18,20,16,12,12,16,13,15,2nd


--------------------------
## Indexing & Filtering 
------------------------

#### Renaming an index: The `.rename()` method

<u> Parameters: </u>
- **mapper**: Dict-like or function transformations to apply to specified axis' values. In case of a function you only need to pass in the name and not call them.
- **axis**: index (0) or, columns(1).

In [46]:
# say, we would like to set the president name as our index and use initial for first name and not the full name
def name_to_initial(val):
    vals = val.split(" ")
    return " ".join([f'{vals[0][0]}.', *vals[1:]])  # unpack the items in the vals[1:] list

siena_2018.set_index("President").rename(name_to_initial).head(3) # or, lambda name_: " ".join([f'{name_.split()[0][0]}.', *name_.split()[1:]]) 

,Seq,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall,Average_rank,Quartile_rank
President,,,,,,,,,,,,,,,,,,,,,,,,,
G. Washington,1,Independent,7,7,1,10,1,6,2,2,1,11,2,18,1,1,1,1,2,2,1,2,1,1,1st
J. Adams,2,Federalist,3,13,4,4,24,14,31,21,21,13,8,28,17,4,13,15,19,13,16,10,14,13,2nd
T. Jefferson,3,Democratic-Republican,2,2,14,1,8,5,14,6,6,4,4,5,5,7,20,4,6,9,7,5,5,5,1st


#### Resetting indexes to monotonically increasing integers: The `.reset_index()` method

In [48]:
siena_2018.set_index("President").reset_index().head(3)

,President,Seq,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall,Average_rank,Quartile_rank
0,George Washington,1,Independent,7,7,1,10,1,6,2,2,1,11,2,18,1,1,1,1,2,2,1,2,1,1,1st
1,John Adams,2,Federalist,3,13,4,4,24,14,31,21,21,13,8,28,17,4,13,15,19,13,16,10,14,13,2nd
2,Thomas Jefferson,3,Democratic-Republican,2,2,14,1,8,5,14,6,6,4,4,5,5,7,20,4,6,9,7,5,5,5,1st


### Indexing by Name: The `.loc[]` method

The **`.loc[row indexer, column indexer]`** attribute is **primarily label based**, but may also be used with a boolean array.

Allowable inputs may be:
- **Scalar:** if any one of the indexer is passed as a scalar, it will return,
    - a dataframe if there are multiple instances and,
    - a series if there's only one entry. This series will have,
        - columns set as index if axis=0.
        - rows set as index if axis=1.

For it to return a dataframe in all cases we have to pass in the scalar as a list.
- **Array of labels**
- **Slice object:** Slicing with .loc includes both the start and end. *Some notes:*
    - If the axis of slicing has unsorted duplicate index labels we will first need to sort the indexes with **.sort_index()**.
    - Slicing with string indexes only works if you sort them.
    - Partial slicing can only be done on string types and not on categorical type.
- **A boolean array:** of the same length as the indexing axis.
- **A callable function:** that returns one of the above.

#### Using functions with .loc (for filtering)

The main advantage of using functions with .loc is that, the function will receive the current state of the dataframe as input. This is specially useful when multiple operations are chained together.

In [102]:
# let us select presidents with average rank < 10 and return first 3 columns of data about them
siena_2018.loc[siena_2018.Average_rank < 10, lambda df_: df_.columns[:3]].head(3)  # :3 as first column is the index column

,Seq,President,Party
1,1,George Washington,Independent
3,3,Thomas Jefferson,Democratic-Republican
4,4,James Madison,Democratic-Republican


In [101]:
# the same can be achieved by the following section of code
siena_2018.loc[siena_2018.Average_rank < 10, "Seq":"Party"].head(3)

,Seq,President,Party
1,1,George Washington,Independent
3,3,Thomas Jefferson,Democratic-Republican
4,4,James Madison,Democratic-Republican


### Indexing by Position: The `.iloc[]` method

The **`.iloc[row indexer, column indexer]`** attribute operates on **indexes and not index labels**. It can also be used with a boolean array.

Allowable inputs may be:
- **Scalar:** if any one of the indexer is passed as a scalar, it will return,
    - a dataframe if there are multiple instances and,
    - a series if there's only one entry. This series will have,
        - columns set as index labels if axis=0.
        - rows set as index labels if axis=1.

For it to return a dataframe in all cases we have to pass in the scalar as a list.
- **Array of indexes**
- **Slice object:** Slicing with .iloc includes only the start and not the end. *Note:*, if the axis being sliced has unsorted duplicate indexed entries we will first need to sort the indexes with **.sort_index()**.
- **A boolean array:** of the same length as the indexing axis.
- **A callable function:** that returns one of the above.

### Filtering with boolean arrays (Boolean Masking)

Boolean arrays can be used to filter data from a dataframe. Using different math operators (such as, &, <, >, | etc.) complex filters can be implemented.

In [106]:
# let's filter out the presidents who was a republican and has an average rank < 10. 
try:
    siena_2018[siena_2018.Average_rank < 10 & siena_2018.Party == "Republican"]
except TypeError as err: print(err)

unsupported operand type(s) for &: 'int' and 'Categorical'


The takeaway is, you should always put parentheses around multiple conditions in index operations if you inline them as some operators has precedence over others.

In [107]:
# now lets do this properly
siena_2018[(siena_2018.Average_rank < 10) & (siena_2018.Party == "Republican")]

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,Overall_ability,Party_leadership,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall,Average_rank,Quartile_rank
16,16,Abraham Lincoln,Republican,28,1,2,2,18,1,1,1,2,1,1,2,4,3,4,2,1,6,2,1,3,3,1st
25,26,Theodore Roosevelt,Republican,5,4,8,6,2,2,15,4,4,5,5,7,7,9,3,5,4,3,5,4,4,4,1st
33,34,Dwight D. Eisenhower,Republican,11,18,5,17,7,21,5,5,5,20,7,15,9,5,6,11,8,7,3,6,6,6,1st


### Filtering with the `.query()` method